In [1]:
import pandas as pd
import syft as sy
import numpy as np
sy.logger.remove()

### Logging into the domain Nodes

In [2]:
# We will login into Canada and Italy domain node
ca = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)
it = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8082)

Connecting to http://localhost:8081... done! 	 Logging into xenodochial_schmidhuber... done!
Connecting to http://localhost:8082... done! 	 Logging into lucid_bostrom... done!


In [3]:
ca.datasets

Idx,Name,Description,Assets,Id
[0],Canada Trade Data - First few rows4,A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Canada Trade""] -> Tensor",2f3bbeb6-6bfc-4cfc-a631-9b87cbf5d49b
[1],USA Trade Data - First few rows,A collection of reports from USA's statistics bureau about how much it thinks it imports and exports from other countries.,"[""USA Trade""] -> Tensor",af819ddf-56bf-4123-9523-9cfb638cea05


In [4]:
it.datasets

Idx,Name,Description,Assets,Id
[0],USA Trade Data - First few rows,A collection of reports from USA's statistics bureau about how much it thinks it imports and exports from other countries.,"[""USA Trade""] -> Tensor",88b08e5d-e164-4099-b2c9-a44fdc01d9c9


In [5]:
ca.privacy_budget

41.592815128439895

In [6]:
it.privacy_budget

41.592815128439895

In [7]:
ca_data = ca.datasets[-1]['USA Trade']
it_data = it.datasets[-1]['USA Trade']

In [8]:
ca_data

<TensorWrappedSingleEntityPhiTensorPointer -> xenodochial_schmidhuber:0ac6fe6ae9e1444bbc109bb67b746510>

In [9]:
it_data

<TensorWrappedSingleEntityPhiTensorPointer -> lucid_bostrom:555fbc77424e4297803273017a4222c5>

In [10]:
from syft.core.tensor.smpc.mpc_tensor import MPCTensor

In [11]:
self = ca_data
other = it_data

In [15]:
parties = [self.client, other.client]
self_mpc = MPCTensor(secret=self, shape=self.public_shape, parties=parties)
other_mpc = MPCTensor(
    secret=other, shape=other.public_shape, parties=parties
)

In [16]:
(
    self_shrs,
    other_shrs,
    kwargs,
    ring_size,
    parties,
    parties_info,
    self_shape,
    other_shape,
) = MPCTensor.prepare_arguments_and_run_checks(self_mpc, other_mpc)

In [21]:
out = self_shrs[0] + other_shrs[0]

In [22]:
out2 = self_shrs[1] + other_shrs[1]

In [28]:
# Step 6: Execute the SMPC operation
# TODO: some complex hooking logic on ShareTensor means we're passing in 'a' twice
res_shares = [a.__add__(b, **kwargs) for a, b in zip(self_shrs, other_shrs)]

In [12]:
result = ca_data + it_data

A:<TensorPointer -> lucid_bostrom:2a873523174947d4837e2a816c83e2b2>
B:<TensorPointer -> lucid_bostrom:c26bfa8936f948cd8829288b56e519d4>


In [13]:
public_result = result.publish(sigma=2)

In [19]:
public_result.block_with_timeout(3).get()

TypeError: block_with_timeout() got an unexpected keyword argument 'secs_per_poll'

In [16]:
ca.privacy_budget

41.592815128439895

In [17]:
it.privacy_budget

41.592815128439895

In [18]:
public_result.get()

array([ 0,  1, -1, -2, -1, -3, -7, -2,  0, -2], dtype=int32)